# <font color = lightcoral>TODS
>    
### <font color = red> New

> Getting Started

## <font color = grey> Admin

In [7]:
import pandas as pd
import os
import gzip
import time
import warnings
import os
import shutil
from datetime import datetime
# Set display option to show the full length of the column
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# Determine the base directory based on the effective user ID
if os.geteuid() == 0:  # root user
    base_dir = '/root'
else:
    base_dir = '/home/sagemaker-user'

# Set the environment variable
os.environ['BASE_DIR'] = base_dir

# Verify the base directory
print(f"Base directory set to: {base_dir}")

Base directory set to: /home/sagemaker-user


# <font color = tomato> Common

In [3]:
# ################################### Parameters and Hyperparameters ################################

# # Parameters and Hyperparameters
# # data_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/10.20240705_StartOver_NewFeatures/01.Full_Base/'
# data_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/10.20240705_StartOver_NewFeatures/03.Sample_Base/'

# # input_file = f"{data_dir}/20240704__full__new_features_v1.10.parquet"
# input_file = f"{data_dir}/20240704__sample__new_features_v1.50.parquet"

# # output_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/10.20240705_StartOver_NewFeatures/02.Full_Train_Test'tput_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS'
# output_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS'

# max_events = 20
# input_length = 10
# gap = 1
# prediction_period = 1
# test_size = 0.2
# shuffle = False

# hidden_size = 64
# num_layers = 2
# output_size = 1
# num_epochs = 50
# batch_size = 16
# learning_rate = 0.001
# dropout = 0.5  # Adding dropout to prevent overfitting

# print("Parameters and Hyperparameters Set")
# print("*"*100 + "\n")

# ################################### Data Preprocessing ################################

# # Data Preprocessing
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.decomposition import PCA

# # Load the data
# df = pd.read_parquet(input_file)

# # Select derived features and a subset of EventID columns
# selected_columns = ['time_start_int', 'Class', 'unique_events', 'most_frequent_event', 'transitions', 'entropy']
# event_id_columns = [col for col in df.columns if col.startswith('EventId_')]

# # Replace -1 values with 0
# df[event_id_columns].replace(-1, 0, inplace=True)

# # Scale numerical features
# scaler = StandardScaler()
# numerical_features = ['unique_events', 'transitions', 'entropy']
# df[selected_columns[2:]] = scaler.fit_transform(df[selected_columns[2:]])

# # Encode categorical features
# label_encoder = LabelEncoder()
# df['most_frequent_event'] = label_encoder.fit_transform(df['most_frequent_event'])

# # Apply PCA to EventID columns to reduce them to 50 features
# pca = PCA(n_components=max_events)
# event_id_pca = pca.fit_transform(df[event_id_columns])

# # Create a new DataFrame with the reduced EventID features
# event_id_pca_df = pd.DataFrame(event_id_pca, columns=[f'EventId_PCA_{i+1}' for i in range(max_events)])

# # Combine the reduced EventID features with the selected columns
# df_reduced = pd.concat([df[selected_columns], event_id_pca_df], axis=1)

# print("Data Preprocessed")
# print("*"*100 + "\n")

# ################################### Create Sequences ################################

# # Function to create non-overlapping sequences
# def create_sequences(data, time_index_col, feature_cols, target_col, input_length=input_length, gap=1, prediction_period=1):
#     sequences = []
#     targets = []
    
#     start_idx = 0
#     while start_idx + input_length + gap + prediction_period <= len(data):
#         end_idx = start_idx + input_length
#         sequence = data[feature_cols].iloc[start_idx:end_idx].values
#         target = data[target_col].iloc[end_idx + gap:end_idx + gap + prediction_period].values[0]
#         sequences.append(sequence)
#         targets.append(target)
#         # start_idx = end_idx + gap + prediction_period  # Move to the next non-overlapping sequence
#         start_idx += 1  # Move to the next row to create overlapping sequences
    
#     return np.array(sequences), np.array(targets)

# # feature_cols = selected_columns + selected_event_ids
# feature_cols = [col for col in df_reduced.columns if col != 'Class']
# target_col = 'Class'

# X, y = create_sequences(df_reduced, 'time_start_int', feature_cols, target_col)

# print("Created Sequences")
# print("*"*100 + "\n")

# ################################### Split & SMOTE ################################

# # Split data into training and testing sets
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=shuffle)

# # Flatten X_train to 2D array for SMOTE
# X_train = X_train.reshape(X_train.shape[0], -1)

# # Apply SMOTE
# smote = SMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)

# # Reshape X_train back to 3D array
# X_train = X_train.reshape(-1, input_length, max_events + 5)

# # Drop the first column from X_train and X_test
# X_train = X_train[:, :, 1:]
# X_test = X_test[:, :, 1:]

# # # Save the datasets
# # np.save(f'{output_dir}/X_train.npy', X_train)
# # np.save(f'{output_dir}/X_test.npy', X_test)
# # np.save(f'{output_dir}/y_train.npy', y_train)
# # np.save(f'{output_dir}/y_test.npy', y_test)

# print("Split, SMOTE & Train/Test Saved")
# print("*"*100 + "\n")

# ################################### Recombine for TODS ################################

# # Flatten X_train and X_test to 2D arrays
# X_train_flat = X_train.reshape(X_train.shape[0], -1)
# X_test_flat = X_test.reshape(X_test.shape[0], -1)

# # Concatenate X and y for train and test sets
# train_data = np.concatenate([X_train_flat, y_train.reshape(-1, 1)], axis=1)
# test_data = np.concatenate([X_test_flat, y_test.reshape(-1, 1)], axis=1)

# # Combine train and test data
# combined_data = np.concatenate([train_data, test_data], axis=0)

# # Create a DataFrame
# column_names = [f'feature_{i+1}' for i in range(X_train_flat.shape[1])] + ['label']
# df_combined = pd.DataFrame(combined_data, columns=column_names)

# # Drop feature_2 (time_index)
# df_combined = df_combined.drop(columns=['feature_2'])

# # Save to CSV
# # csv_output_path = f'{output_dir}/c20240705_TODS_full_v1.00.csv'
# csv_output_path = f'{output_dir}/c20240705_TODS_sample_v1.00.csv'
# df_combined.to_csv(csv_output_path, index=False)

# # Save to Parquet
# # parquet_output_path = f'{output_dir}/c20240705_TODS_full_v1.00.parquet'
# parquet_output_path = f'{output_dir}/c20240705_TODS_sample_v1.00.parquet'

# df_combined.to_parquet(parquet_output_path, index=False)

# print(f"Combined data saved to CSV at {csv_output_path}")
# print(f"Combined data saved to Parquet at {parquet_output_path}")

Parameters and Hyperparameters Set
****************************************************************************************************

Data Preprocessed
****************************************************************************************************

Created Sequences
****************************************************************************************************

Split, SMOTE & Train/Test Saved
****************************************************************************************************

Combined data saved to CSV at /home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/c20240705_TODS_sample_v1.00.csv
Combined data saved to Parquet at /home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/c20240705_TODS_sample_v1.00.parquet


### <font color = grey> Review generated files

In [4]:
# # Display the first 5 lines of the CSV file
# df_csv = pd.read_csv(csv_output_path)
# print("First 5 lines of the CSV file:")
# display(df_csv.head())

# # Display the first 5 lines of the Parquet file
# df_parquet = pd.read_parquet(parquet_output_path)
# print("First 5 lines of the Parquet file:")
# display(df_parquet.head())

First 5 lines of the CSV file:


,feature_1,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,label
0,-0.449756,-0.071532,-0.935017,-511.899229,-1897.728545,1892.193864,-1800.512951,1716.153147,108.966949,-1087.335503,711.265902,-414.147623,-690.902204,-295.074213,119.688772,-574.414342,18.642667,-334.981288,633.789239,256.889949,235.119026,-61.852877,-85.578348,-0.256462,36.0,-0.067848,-0.927084,2553.585704,-4906.251012,2335.325503,-314.707397,-1356.950060,-898.198017,-1147.085089,418.328739,371.014384,1076.276228,193.553592,-479.117805,746.671204,-41.674053,1331.949218,-710.600821,155.939896,995.075728,-353.385971,150.517038,-0.063168,36.0,-0.045745,-0.825372,1792.713630,-4293.303647,2530.837323,-1035.194496,-299.344505,-731.962749,-2155.028632,1078.906461,4.759914,1330.074696,1135.415179,-87.209520,695.512698,-68.084209,537.768010,-433.176891,77.316255,614.537327,-188.355901,152.214179,-0.449756,36.0,-0.071532,-0.935017,377.955669,-2877.793346,2271.378098,-1653.789303,1059.765109,-264.703657,-2030.153238,1164.539307,-329.725082,541.745058,1003.288816,274.968400,159.332289,-12.997391,-708.439256,457.168845,-4.379625,-362.414328,137.474288,-114.283946,0.516713,36.0,-0.060481,-0.677433,-1013.887299,-1306.810880,1576.879362,-1730.362375,1888.873593,287.655745,-303.580668,291.213129,-339.514773,-1138.600817,-963

First 5 lines of the Parquet file:


,feature_1,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,label
0,-0.449756,-0.071532,-0.935017,-511.899229,-1897.728545,1892.193864,-1800.512951,1716.153147,108.966949,-1087.335503,711.265902,-414.147623,-690.902204,-295.074213,119.688772,-574.414342,18.642667,-334.981288,633.789239,256.889949,235.119026,-61.852877,-85.578348,-0.256462,36.0,-0.067848,-0.927084,2553.585704,-4906.251012,2335.325503,-314.707397,-1356.950060,-898.198017,-1147.085089,418.328739,371.014384,1076.276228,193.553592,-479.117805,746.671204,-41.674053,1331.949218,-710.600821,155.939896,995.075728,-353.385971,150.517038,-0.063168,36.0,-0.045745,-0.825372,1792.713630,-4293.303647,2530.837323,-1035.194496,-299.344505,-731.962749,-2155.028632,1078.906461,4.759914,1330.074696,1135.415179,-87.209520,695.512698,-68.084209,537.768010,-433.176891,77.316255,614.537327,-188.355901,152.214179,-0.449756,36.0,-0.071532,-0.935017,377.955669,-2877.793346,2271.378098,-1653.789303,1059.765109,-264.703657,-2030.153238,1164.539307,-329.725082,541.745058,1003.288816,274.968400,159.332289,-12.997391,-708.439256,457.168845,-4.379625,-362.414328,137.474288,-114.283946,0.516713,36.0,-0.060481,-0.677433,-1013.887299,-1306.810880,1576.879362,-1730.362375,1888.873593,287.655745,-303.580668,291.213129,-339.514773,-1138.600817,-963

### <font color = grey> Final Conversions
> #1 convert all -ve values to +ve (ie abs)
>
> #2 normalize all values>1  except label

In [7]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# # Load the Parquet file
# parquet_output_path = f'{output_dir}/c20240705_TODS_sample_v1.00.parquet'
# df_parquet = pd.read_parquet(parquet_output_path)

# # Get the feature columns excluding the label
# feature_columns = [col for col in df_parquet.columns if col != 'label']

# # Process the data in steps
# # 1. Convert all negative values to positive
# df_parquet[feature_columns] = df_parquet[feature_columns].applymap(lambda x: abs(x))

# # 2. Normalize/scale values greater than 1
# scaler = MinMaxScaler()
# for col in feature_columns:
#     if df_parquet[col].max() > 1:
#         df_parquet[col] = scaler.fit_transform(df_parquet[[col]])

# # Save the modified data to CSV and Parquet
# csv_output_path = f'{output_dir}/c20240705_TODS_sample_normalized_v1.01.csv'
# df_parquet.to_csv(csv_output_path, index=False)

# parquet_output_path = f'{output_dir}/c20240705_TODS_sample_normalized_v1.01.parquet'
# df_parquet.to_parquet(parquet_output_path, index=False)

# print(f"Modified data saved to CSV at {csv_output_path}")
# print(f"Modified data saved to Parquet at {parquet_output_path}")

Modified data saved to CSV at /home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/c20240705_TODS_sample_normalized_v1.01.csv
Modified data saved to Parquet at /home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/c20240705_TODS_sample_normalized_v1.01.parquet


### <font color = grey> Review generated files

In [8]:
# # output_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/10.20240705_StartOver_NewFeatures/02.Full_Train_Test'tput_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS'
output_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS'

# Define the file paths
csv_output_path = f'{output_dir}/c20240705_TODS_sample_normalized_v1.01.csv'
parquet_output_path = f'{output_dir}/c20240705_TODS_sample_normalized_v1.01.parquet'

# Display the first 5 lines of the CSV file
df_csv = pd.read_csv(csv_output_path)
print(f"First 5 lines of the CSV file ({csv_output_path}):")
display(df_csv.head())

# Get the number of columns in the CSV file
num_columns_csv = df_csv.shape[1]
print(f"Total number of columns in the CSV file: {num_columns_csv}")

# Display the first 5 lines of the Parquet file
df_parquet = pd.read_parquet(parquet_output_path)
print(f"First 5 lines of the Parquet file ({parquet_output_path}):")
display(df_parquet.head())

First 5 lines of the CSV file (/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/c20240705_TODS_sample_normalized_v1.01.csv):


,feature_1,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,label
0,0.027476,0.002386,0.178079,0.005292,0.041661,0.055649,0.044455,0.073619,0.004112,0.052112,0.018805,0.032562,0.079432,0.033424,0.010339,0.098758,0.001104,0.050610,0.090852,0.024460,0.019789,0.002325,0.003043,0.015665,0.266667,0.002262,0.176608,0.026408,0.107729,0.068677,0.007766,0.058209,0.033893,0.054976,0.011060,0.029171,0.123741,0.021925,0.041394,0.128377,0.002469,0.201240,0.101864,0.014849,0.083759,0.013282,0.005352,0.003850,0.266667,0.001527,0.157194,0.018484,0.094272,0.074419,0.025562,0.012839,0.027620,0.103266,0.028527,0.000373,0.152923,0.128628,0.007534,0.119573,0.004034,0.081249,0.062096,0.007362,0.051727,0.007079,0.005413,0.027476,0.266667,0.002383,0.178121,0.003878,0.063197,0.066783,0.040832,0.045460,0.009988,0.097294,0.030791,0.025925,0.062284,0.113646,0.023756,0.027391,0.000770,0.107031,0.065534,0.000413,0.030505,0.005167,0.004064,0.031565,0.266667,0.002016,0.129002,0.010440,0.028693,0.046375,0.042723,0.081029,0.010854,0.014549,0.007699,0.026694,0.130908,0.109106,0.006286,0.076708,0.001944,0.041418,0.002068,0.010798,0.030378,0.005748,0.003397,0.015669,0.985185,0.002262,0.007139,0.026741,0.018610,0.012327,0.008068,0.014894,0.001683,0.003846,0.000252,0.000505,0.005305,0.004447,0.000213,0.065063,0.000

Total number of columns in the CSV file: 240
First 5 lines of the Parquet file (/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/c20240705_TODS_sample_normalized_v1.01.parquet):


,feature_1,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,label
0,0.027476,0.002386,0.178079,0.005292,0.041661,0.055649,0.044455,0.073619,0.004112,0.052112,0.018805,0.032562,0.079432,0.033424,0.010339,0.098758,0.001104,0.050610,0.090852,0.024460,0.019789,0.002325,0.003043,0.015665,0.266667,0.002262,0.176608,0.026408,0.107729,0.068677,0.007766,0.058209,0.033893,0.054976,0.011060,0.029171,0.123741,0.021925,0.041394,0.128377,0.002469,0.201240,0.101864,0.014849,0.083759,0.013282,0.005352,0.003850,0.266667,0.001527,0.157194,0.018484,0.094272,0.074419,0.025562,0.012839,0.027620,0.103266,0.028527,0.000373,0.152923,0.128628,0.007534,0.119573,0.004034,0.081249,0.062096,0.007362,0.051727,0.007079,0.005413,0.027476,0.266667,0.002383,0.178121,0.003878,0.063197,0.066783,0.040832,0.045460,0.009988,0.097294,0.030791,0.025925,0.062284,0.113646,0.023756,0.027391,0.000770,0.107031,0.065534,0.000413,0.030505,0.005167,0.004064,0.031565,0.266667,0.002016,0.129002,0.010440,0.028693,0.046375,0.042723,0.081029,0.010854,0.014549,0.007699,0.026694,0.130908,0.109106,0.006286,0.076708,0.001944,0.041418,0.002068,0.010798,0.030378,0.005748,0.003397,0.015669,0.985185,0.002262,0.007139,0.026741,0.018610,0.012327,0.008068,0.014894,0.001683,0.003846,0.000252,0.000505,0.005305,0.004447,0.000213,0.065063,0.000

___
# <font color = tomato> TODS

---

In [12]:
import pandas as pd

from tods import schemas as schemas_utils
from tods.utils import generate_dataset_problem, evaluate_pipeline

# table_path = 'datasets/yahoo_sub_5.csv'

table_path = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/yahoo_sub_5.csv'
# table_path = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/11.20240705_TODS/c20240705_TODS_sample_normalized_v1.01.csv'
target_index = 240 # what column is the target
metric = 'F1_MACRO' # F1 on both label 0 and 1

# Read data and generate dataset and problem
df = pd.read_csv(table_path)
dataset, problem_description = generate_dataset_problem(df, target_index)

# Load the default pipeline
pipeline = schemas_utils.load_default_pipeline()

# Run the pipeline
pipeline_result = evaluate_pipeline(dataset, pipeline, metric)

InvalidArgumentTypeError: Structural type is not a subclass of a sequence.

___
## <font color = grey> GRU

---

In [ ]:
################################### GRU ################################

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the GRU model with Dropout
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = GRUModel(max_events + 4, hidden_size, num_layers, output_size, dropout)
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
print("Model Trained")
print("*"*100 + "\n")

################################### Model Eval ################################

from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_outputs = torch.sigmoid(test_outputs)  # Apply sigmoid to get probabilities
    test_outputs = test_outputs.cpu().numpy()

    # Convert y_test_tensor to NumPy array only if it is a tensor
    if isinstance(y_test_tensor, torch.Tensor):
        y_test_tensor = y_test_tensor.cpu().numpy()

    test_preds = (test_outputs > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate evaluation metrics
conf_matrix = confusion_matrix(y_test_tensor, test_preds)
precision = precision_score(y_test_tensor, test_preds)
recall = recall_score(y_test_tensor, test_preds)
accuracy = accuracy_score(y_test_tensor, test_preds)
f1 = f1_score(y_test_tensor, test_preds)

print("Confusion Matrix:\n", conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Visualize the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

---

## <font color = grey> Single - 1D-CNN

---

In [ ]:
################################### 1D CNN ################################

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Define the 1D CNN model with Dropout
class CNN1DModel(nn.Module):
    def __init__(self, input_size, num_classes, dropout=0.5):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        # Calculate the correct size after convolutions and pooling
        self.fc1_input_size = 128 * (input_length // 2 // 2)
        self.fc1 = nn.Linear(self.fc1_input_size, 128)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = x.transpose(1, 2)  # Transpose to [batch_size, input_size, sequence_length]
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = CNN1DModel(max_events + 4, output_size, dropout)
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
print("Model Trained")
print("*"*100 + "\n")

################################### Model Eval ################################

from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_outputs = torch.sigmoid(test_outputs)  # Apply sigmoid to get probabilities
    test_outputs = test_outputs.cpu().numpy()

    # Convert y_test_tensor to NumPy array only if it is a tensor
    if isinstance(y_test_tensor, torch.Tensor):
        y_test_tensor = y_test_tensor.cpu().numpy()

    test_preds = (test_outputs > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate evaluation metrics
conf_matrix = confusion_matrix(y_test_tensor, test_preds)
precision = precision_score(y_test_tensor, test_preds)
recall = recall_score(y_test_tensor, test_preds)
accuracy = accuracy_score(y_test_tensor, test_preds)
f1 = f1_score(y_test_tensor, test_preds)

print("Confusion Matrix:\n", conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Visualize the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

## <font color = grey> Single - Transformer Model

---

In [ ]:
################################### Transformer ################################

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        self.pos_encoder = nn.Embedding(input_length, hidden_size)
        self.transformer = nn.Transformer(hidden_size, nhead=4, num_encoder_layers=num_layers, num_decoder_layers=num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.embedding(x)
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0)
        x = x + self.pos_encoder(positions)
        x = self.transformer(x, x)
        x = self.fc(x[:, -1, :])
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = TransformerModel(max_events + 4, hidden_size, num_layers, output_size, dropout)
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
print("Model Trained")
print("*"*100 + "\n")

################################### Model Eval ################################

from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_outputs = torch.sigmoid(test_outputs)  # Apply sigmoid to get probabilities
    test_outputs = test_outputs.cpu().numpy()

    # Convert y_test_tensor to NumPy array only if it is a tensor
    if isinstance(y_test_tensor, torch.Tensor):
        y_test_tensor = y_test_tensor.cpu().numpy()

    test_preds = (test_outputs > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate evaluation metrics
conf_matrix = confusion_matrix(y_test_tensor, test_preds)
precision = precision_score(y_test_tensor, test_preds)
recall = recall_score(y_test_tensor, test_preds)
accuracy = accuracy_score(y_test_tensor, test_preds)
f1 = f1_score(y_test_tensor, test_preds)

print("Confusion Matrix:\n", conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Visualize the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# <font color = tomato> For Later

## <font color = grey> Loop

In [ ]:
# ################################### Parameters and Hyperparameters ################################

# # Parameters and Hyperparameters
# data_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/10.20240705_StartOver_NewFeatures/01.Full_Base/'
# input_file = f"{data_dir}/20240704__full__new_features_v1.10.parquet"
# output_dir = '/home/ubuntu/efs-w210-capstone-ebs/11.Data/01.BGL/10.20240705_StartOver_NewFeatures/02.Full_Train_Test'

# max_events_list = [10, 20]  # Different values to test
# input_length_list = [10, 20]  # Different values to test
# hidden_size_list = [64, 128]  # Different values to test
# dropout_list = [0.3, 0.5]  # Different values to test

# gap = 1
# prediction_period = 1
# test_size = 0.2
# shuffle = False

# num_layers = 2
# output_size = 1
# num_epochs = 50
# batch_size = 16
# learning_rate = 0.001

# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.decomposition import PCA
# from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# import matplotlib.pyplot as plt
# import seaborn as sns
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import train_test_split

# # DataFrame to track results
# results_df = pd.DataFrame(columns=['Run', 'Precision', 'Recall', 'Accuracy', 'F1', 'TN', 'FP', 'FN', 'TP'])

# run_id = 0

# for max_events in max_events_list:
#     for input_length in input_length_list:
#         for hidden_size in hidden_size_list:
#             for dropout in dropout_list:
#                 run_id += 1
#                 print(f"Run {run_id}: max_events={max_events}, input_length={input_length}, hidden_size={hidden_size}, dropout={dropout}")
                
#                 ################################### Data Preprocessing ################################

#                 # Load the data
#                 df = pd.read_parquet(input_file)

#                 # Select derived features and a subset of EventID columns
#                 selected_columns = ['time_start_int', 'Class', 'unique_events', 'most_frequent_event', 'transitions', 'entropy']
#                 event_id_columns = [col for col in df.columns if col.startswith('EventId_')]

#                 # Replace -1 values with 0
#                 df[event_id_columns].replace(-1, 0, inplace=True)

#                 # Scale numerical features
#                 scaler = StandardScaler()
#                 numerical_features = ['unique_events', 'transitions', 'entropy']
#                 df[selected_columns[2:]] = scaler.fit_transform(df[selected_columns[2:]])

#                 # Encode categorical features
#                 label_encoder = LabelEncoder()
#                 df['most_frequent_event'] = label_encoder.fit_transform(df['most_frequent_event'])

#                 # Apply PCA to EventID columns to reduce them to 50 features
#                 pca = PCA(n_components=max_events)
#                 event_id_pca = pca.fit_transform(df[event_id_columns])

#                 # Create a new DataFrame with the reduced EventID features
#                 event_id_pca_df = pd.DataFrame(event_id_pca, columns=[f'EventId_PCA_{i+1}' for i in range(max_events)])

#                 # Combine the reduced EventID features with the selected columns
#                 df_reduced = pd.concat([df[selected_columns], event_id_pca_df], axis=1)

#                 ################################### Create Sequences ################################

#                 # Function to create non-overlapping sequences
#                 def create_sequences(data, time_index_col, feature_cols, target_col, input_length, gap=1, prediction_period=1):
#                     sequences = []
#                     targets = []
                    
#                     start_idx = 0
#                     while start_idx + input_length + gap + prediction_period <= len(data):
#                         end_idx = start_idx + input_length
#                         sequence = data[feature_cols].iloc[start_idx:end_idx].values
#                         target = data[target_col].iloc[end_idx + gap:end_idx + gap + prediction_period].values[0]
#                         sequences.append(sequence)
#                         targets.append(target)
#                         start_idx = end_idx + gap + prediction_period  # Move to the next non-overlapping sequence
                    
#                     return np.array(sequences), np.array(targets)

#                 # feature_cols = selected_columns + selected_event_ids
#                 feature_cols = [col for col in df_reduced.columns if col != 'Class']
#                 target_col = 'Class'

#                 X, y = create_sequences(df_reduced, 'time_start_int', feature_cols, target_col, input_length=input_length)

#                 ################################### Split & SMOTE ################################

#                 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=shuffle)

#                 # Flatten X_train to 2D array for SMOTE
#                 X_train = X_train.reshape(X_train.shape[0], -1)

#                 # Apply SMOTE
#                 smote = SMOTE(random_state=42)
#                 X_train, y_train = smote.fit_resample(X_train, y_train)

#                 # Reshape X_train back to 3D array
#                 X_train = X_train.reshape(-1, input_length, max_events + 5)

#                 # Drop the first column from X_train and X_test
#                 X_train = X_train[:, :, 1:]
#                 X_test = X_test[:, :, 1:]

#                 ################################### LSTM ################################

#                 # Define the LSTM model with Dropout
#                 class LSTMModel(nn.Module):
#                     def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
#                         super(LSTMModel, self).__init__()
#                         self.hidden_size = hidden_size
#                         self.num_layers = num_layers
#                         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
#                         self.fc = nn.Linear(hidden_size, output_size)
                    
#                     def forward(self, x):
#                         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#                         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#                         out, _ = self.lstm(x, (h0, c0))
#                         out = self.fc(out[:, -1, :])
#                         return out

#                 # Convert data to PyTorch tensors
#                 X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
#                 y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
#                 X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#                 y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

#                 # Create DataLoader
#                 train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
#                 train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#                 # Initialize the model, loss function, and optimizer
#                 model = LSTMModel(max_events + 4, hidden_size, num_layers, output_size, dropout)
#                 criterion = nn.BCEWithLogitsLoss()  # For binary classification
#                 optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#                 # Training loop
#                 for epoch in range(num_epochs):
#                     model.train()
#                     for X_batch, y_batch in train_loader:
#                         outputs = model(X_batch)
#                         loss = criterion(outputs, y_batch)
                        
#                         optimizer.zero_grad()
#                         loss.backward()
#                         optimizer.step()
                    
#                     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
                    
#                 ################################### Model Eval ################################

#                 # Evaluate the model on the test set
#                 model.eval()
#                 with torch.no_grad():
#                     test_outputs = model(X_test_tensor)
#                     test_outputs = torch.sigmoid(test_outputs)  # Apply sigmoid to get probabilities
#                     test_outputs = test_outputs.cpu().numpy()

#                     # Convert y_test_tensor to NumPy array only if it is a tensor
#                     if isinstance(y_test_tensor, torch.Tensor):
#                         y_test_tensor = y_test_tensor.cpu().numpy()

#                     test_preds = (test_outputs > 0.5).astype(int)  # Convert probabilities to binary predictions

#                 # Calculate evaluation metrics
#                 conf_matrix = confusion_matrix(y_test_tensor, test_preds)
#                 precision = precision_score(y_test_tensor, test_preds)
#                 recall = recall_score(y_test_tensor, test_preds)
#                 accuracy = accuracy_score(y_test_tensor, test_preds)
#                 f1 = f1_score(y_test_tensor, test_preds)
#                 tn, fp, fn, tp = conf_matrix.ravel()

#                 # Append results to DataFrame
#                 results_df = pd.concat([results_df, pd.DataFrame([{
#                     'Run': run_id,
#                     'Precision': precision,
#                     'Recall': recall,
#                     'Accuracy': accuracy,
#                     'F1': f1,
#                     'TN': tn,
#                     'FP': fp,
#                     'FN': fn,
#                     'TP': tp
#                 }])], ignore_index=True)

#                 print(f"Run {run_id} completed")
#                 print("*"*100 + "\n")

# ################################### Plot Results ################################

# # Plot the results
# fig, axes = plt.subplots(4, 2, figsize=(15, 20))
# fig.suptitle('Model Performance Metrics Over Different Runs')

# metrics = ['Precision', 'Recall', 'Accuracy', 'F1', 'TN', 'FP', 'FN', 'TP']
# for i, metric in enumerate(metrics):
#     ax = axes[i//2, i%2]
#     ax.plot(results_df['Run'], results_df[metric])
#     ax.set_title(metric)
#     ax.set_xlabel('Run')
#     ax.set_ylabel(metric)

# plt.tight_layout()
# plt.show()